In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt       # чтобы выводить промежуточные фото в jupyter

# Список всех настроечных параметров/констант
WORK_DIR = 'pass_photos'
TEMP_DIR = 'pass_temp'
# TEST_FILE = 'pass_photos/1.jpeg'
IMG_HEIGHT = 1000            # требуемый размер фото для нормализации всех изображений
IMG_WIDTH = 600              # т.к. в задачу входит прочитать только ФИО, обрезаю серию/номер чтобы не усложнять распознавание
INDENT_LEFT = 200            # обрезаем фото т.к. без него получается лучше разделить фото на куски текста
INDENT_BOTTOM = 260          # обрезаем нижние поля
INDENT_TOP = 40              # обрезаем лишнюю часть паспорта снизу

In [2]:
# Функция для получения списка файлов из каталога с фотографиями (как в task_1 и task_2)
def get_files(directory: str) -> list:
    names = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpeg") or filename.endswith(".jpg") or filename.endswith(".png"):
            names.append(os.path.join(directory, filename))

    return names

In [3]:
# Изменение размера
def scale_image(image, scale):     # принимаем объект изображения OpenCV
    
    # получаем текущий размер, вычисляем искомый и создаем измененное изображение
    height, width = image.shape[0], image.shape[1]
    img_width = int(width * scale)
    img_height = int(height * scale)
    img = cv2.resize(image, (img_width, img_height))
    
    return img

In [8]:
# Нормализация размеров и вырезка нужной части паспорта для обработки
def normalize_size(image):     # принимаем объект изображения OpenCV
    
    # нормализуем фото к нужному размеру
    old_height = image.shape[0]     # получаем исходную высоту
    resize_scale = IMG_HEIGHT / old_height       # считаем коэффициент масштабирования изображения до требуемого
    img = scale_image(image=image, scale=resize_scale)
    new_width = img.shape[1]      # получаем новую ширину
    
    # обрезаем паспорт до страницы с фото
    x0 = INDENT_LEFT                            # отступ слева, т.к. корочка и фото нам не важны
    y0 = IMG_HEIGHT // 2 + INDENT_TOP           # обрезка сверху, т.к. верхняя страница с местом выдачи нам не важна 
    x1 = new_width if new_width < IMG_WIDTH else IMG_WIDTH     # обрезаем все лишнее справа, если есть разворот с пропиской
    y1 = IMG_HEIGHT - INDENT_BOTTOM
    img = img[y0:y1, x0:x1]              # обресанный кусок изображения
    
    return img

In [5]:
# Подготовка изображений для распознавания текста
def normalize_color(image):         # принимаем объект изображения OpenCV
    
    # обесцвечиваем и пытаемся снизить шум с помощью размытия
    if len(image.shape) > 2:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)    # преобразуем в ЧБ
    else:
        gray = image
    # blur = cv2.GaussianBlur(gray, (5,5), 0)         # коэффициент размытия подобран вручную
    
    
    # одно изображение используем для распознавания блоков текста. очередность преобраозвания найдена методом тыка
    kernel = np.ones((5,5), 'uint8')
    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    img_block = cv2.erode(gray, kernel, iterations=1)
    #img_block = cv2.dilate(img_block, kernel, iterations=1)
    _, img_block = cv2.threshold(img_block, 0, 255, cv2.THRESH_OTSU, cv2.THRESH_BINARY_INV)
    img_block = cv2.morphologyEx(img_block, cv2.MORPH_OPEN, kernel, iterations=1)
    # img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    
    
    # второе изображение используем для распознавания букв внутри блоков
    #imghsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    #imghsv[:,:,2] = [[max(pixel - 25, 0) if pixel < 190 else min(pixel + 25, 255) for pixel in row] for row in imghsv[:,:,2]]
    #contrast = cv2.cvtColor(imghsv, cv2.COLOR_HSV2BGR)
    # gray = cv2.cvtColor(contrast, cv2.COLOR_BGR2GRAY)    # преобразуем в ЧБ
    
    # при коэффициенте 3 - лучше распознается Васлевский, при 5 - Соколов и Юмакаева
    # img_symbol = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 3, 2)
    # img_symbol = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 3, 2)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_TOZERO+cv2.THRESH_BINARY)
    
    

    return img_block, gray      # Возвращаем контрастную картинку с разбивкой на блоки и простое ЧБ изображение

In [6]:
# Выделяем элементы текста из изображения
def search_blocks(image, limit: int):
    
    height, width = image.shape
    # получаем контуры больших пятен на изображении, внутри которых спрятан текст
    contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    # contours, hierarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # output = image.copy()      #TODO - можно удалить
    
    print(f'Count of Block counoturs: {len(contours)}')
    blocks = []
    for idx, contour in enumerate(contours):
        (x, y, w, h) = cv2.boundingRect(contour)
        print("R", x, y, w, h, hierarchy[0][idx])
        # hierarchy[i][0]: следующий контур текущего уровня
        # hierarchy[i][1]: предыдущий контур текущего уровня
        # hierarchy[i][2]: первый вложенный элемент
        # hierarchy[i][3]: родительский элемент
        # if hierarchy[0][idx][3] == 0:               # если элемент не является самым крупным
        cv2.rectangle(image, (x, y), (x + w, y + h), (70, 0, 0), 1) # для отображаемой картинки
        if limit < h < height and limit < w < width:    # игнорируем маленькие блоки, а также блок размером с изображение
            block = image[y:y + h, x:x + w]     # вырезаем найденный блок из изображения
            blocks.append((idx, y, h, x, w, block))  # сохраняем габариты и изображение блока в список блоков
            
    #print(blocks)
    return blocks
    # return image

In [10]:
# Запускаем цикл по всем фото в рабочей папке
passports = get_files(WORK_DIR)
# col = 5    # количество колонок таблицы
words = []
for i, p in enumerate(passports):     # идем по списку путей к изображениям
    temp_dir = os.path.join(TEMP_DIR, str(i))
    if not os.path.exists(temp_dir):
        os.mkdir(temp_dir)
        
    print(f'==== Image {i}.jpg =====')
    image = normalize_size(cv2.imread(p))     # получаем кусок паспорта с ФИО
    img_blocks, img_gray = normalize_color(image=image)
    
    cv2.imwrite(f'{TEMP_DIR}/{i}_blocs.jpg', img_blocks)
    cv2.imwrite(f'{TEMP_DIR}/{i}_symbols.jpg', img_gray)
    
    words = search_blocks(image=img_blocks, limit=15)
    print(f'Count of words: {len(words)}')
    
    # получаем все обнаруженные слова из файла, в котором читаются символы
    try:
        for word in words[-3:]:        # забираем только последние 3 слова, т.к. там содержатся искомые данные
            y, h, x, w = word[1:5]     # из списка обнаруженных блоков текста забираем координаты этих блоков
            img_word = img_gray[y:y + h, x:x + w]     # вырезаем слово по его координатам
            img_word = scale_image(img_word, 8)        # увеличиваем изображение
            cv2.imwrite(os.path.join(temp_dir, f'{word[0]}.jpg'), img_word)
            
            
            word_blocks, word_gray = normalize_color(image=img_word)
            symbols = search_blocks(image=word_blocks, limit=7)
            for symbol in symbols:
                word_dir = os.path.join(temp_dir, str(word[0]))
                if not os.path.exists(word_dir):
                    os.mkdir(word_dir)
                
                y, h, x, w = symbol[1:5]
                img_symbol = word_gray[y:y + h, x:x + w]
                cv2.imwrite(os.path.join(word_dir, f'{symbol[0]}.jpg'), img_symbol)
            print(f'Count of symbols: {len(symbols)}')
            
            
    except IndexError:             # хотя сейчас Питон не выдает исключение, даже если в списке всего 1 элемент
        print("В паспорте найдено недостаточно данных. Попробуйте сделать более качественный скан.")
    
    
    #plt.subplot(len(passports)//col+1, col, i+1)    # высчитываем высоту таблицы от количества колонок и общего размера списка
    #plt.imshow(image, 'gray')
    # plt.title(p)
    #plt.xticks([]),plt.yticks([])
#plt.show()

==== Image 0.jpg =====
Count of Block counoturs: 13
R 0 0 400 200 [-1 -1  1 -1]
R 45 185 33 12 [ 2 -1 -1  0]
R 44 161 21 13 [ 3  1 -1  0]
R 77 154 59 21 [ 4  2 -1  0]
R 307 151 60 22 [ 5  3 -1  0]
R 220 151 85 23 [ 6  4 -1  0]
R 146 149 50 27 [ 7  5 -1  0]
R 44 126 47 12 [ 8  6 -1  0]
R 225 112 63 22 [ 9  7 -1  0]
R 44 90 25 12 [10  8 -1  0]
R 199 75 107 23 [11  9 -1  0]
R 44 18 47 13 [12 10 -1  0]
R 211 5 77 21 [-1 11 -1  0]
Count of words: 7
Count of Block counoturs: 7
R 0 0 504 176 [-1 -1  1 -1]
R 253 25 117 124 [ 3 -1  2  0]
R 283 52 58 65 [-1 -1 -1  1]
R 22 24 235 135 [6 1 4 0]
R 79 54 18 58 [ 5 -1 -1  3]
R 122 52 44 71 [-1  4 -1  3]
R 378 23 103 123 [-1  3 -1  0]
Count of symbols: 6
Count of Block counoturs: 11
R 0 0 856 184 [-1 -1  1 -1]
R 137 26 114 122 [ 3 -1  2  0]
R 166 52 59 64 [-1 -1 -1  1]
R 603 23 114 122 [5 1 4 0]
R 645 95 38 21 [-1 -1 -1  3]
R 261 23 334 122 [7 3 6 0]
R 421 70 12 16 [-1 -1 -1  5]
R 714 22 118 139 [9 5 8 0]
R 772 56 18 57 [-1 -1 -1  7]
R 21 22 117 139 [

Count of Block counoturs: 11
R 0 0 848 192 [-1 -1  1 -1]
R 135 46 115 124 [ 4 -1  2  0]
R 171 116 46 40 [ 3 -1 -1  1]
R 172 65 36 36 [-1  2 -1  1]
R 21 46 110 125 [6 1 5 0]
R 64 65 43 33 [-1 -1 -1  4]
R 372 45 107 125 [ 7  4 -1  0]
R 250 45 120 125 [ 8  6 -1  0]
R 483 23 347 147 [-1  7  9  0]
R 755 63 29 49 [10 -1 -1  8]
R 704 63 20 88 [-1  9 -1  8]
Count of symbols: 10
Count of Block counoturs: 10
R 0 0 768 168 [-1 -1  1 -1]
R 519 26 110 125 [ 3 -1  2  0]
R 563 44 43 34 [-1 -1 -1  1]
R 276 23 116 124 [5 1 4 0]
R 316 40 42 36 [-1 -1 -1  3]
R 398 22 115 125 [7 3 6 0]
R 441 41 42 40 [-1 -1 -1  5]
R 636 20 112 124 [ 8  5 -1  0]
R 149 19 122 127 [ 9  7 -1  0]
R 21 19 118 124 [-1  8 -1  0]
Count of symbols: 9
==== Image 8.jpg =====
Count of Block counoturs: 13
R 210 195 7 5 [ 1 -1 -1 -1]
R 0 0 400 200 [-1  0  2 -1]
R 40 156 21 13 [ 3 -1 -1  1]
R 73 149 59 21 [ 4  2 -1  1]
R 303 146 60 22 [ 5  3 -1  1]
R 216 146 85 23 [ 6  4 -1  1]
R 142 144 50 28 [ 7  5 -1  1]
R 39 122 48 11 [ 8  6 -1  1]
R